In [82]:
# --- Initial Set Up ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from hmmlearn import hmm


In [83]:

START_DATE = "2015-01-01"
END_DATE = "2026-01-01"
YF_TICKERS = ["^GSPC", "^FTSE", "^990100-USD-STRD"]
TIME_INTERVAL = '1d'
VOL_WINDOW = 30
MOM_WINDOW = 90 
MA_WINDOW = 20 

def process_index(ticker):
    print(f"\n--- Processing {ticker} ---")

    data = yf.download(ticker, start=START_DATE, end=END_DATE, interval=TIME_INTERVAL)
    data = data['Close'].dropna().to_frame()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    
    data['Volatility'] = data['Log_Returns'].rolling(window=VOL_WINDOW).std() * np.sqrt(252)
    
    data['Momentum'] = data['Close'].shift(1) / data['Close'].shift(MOM_WINDOW)

    data['Moving_Avg'] = data['Close'].rolling(window=MA_WINDOW).mean()
    data['Mean_Reversion_Zt'] = data['Close'] - data['Moving_Avg']

    data = data.dropna()

    X = data[['Log_Returns', 'Volatility']].values

    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=100, random_state=42)
    model.fit(X)

    data['Regime'] = model.predict(X)

    regime_means = data.groupby('Regime')['Log_Returns'].mean()
    
    if regime_means[0] > regime_means[1]:
        bull_regime, bear_regime = 0, 1
    else:
        bull_regime, bear_regime = 1, 0

    data['Regime_Label'] = data['Regime'].map({bull_regime: 'Bull', bear_regime: 'Bear'})
    
    print(f"Regime parameters for {ticker}:")
    print(f"  Bull Mean Return: {regime_means[bull_regime]:.4f}, Bull Volatility: {data[data['Regime'] == bull_regime]['Volatility'].mean():.4f}")
    print(f"  Bear Mean Return: {regime_means[bear_regime]:.4f}, Bear Volatility: {data[data['Regime'] == bear_regime]['Volatility'].mean():.4f}")

    return data, bull_regime, bear_regime

def form_trading_rules_and_backtest(data, bull_regime, bear_regime):
        
    data['Buy_Signal'] = (data['Momentum'] > 1) & (data['Regime'] == bull_regime)
    data['Sell_Signal'] = (data['Mean_Reversion_Zt'] < 0) & (data['Regime'] == bear_regime)

    data['Strategy_Returns'] = 0.0
    
    buy_signals = data['Buy_Signal'].shift(1).fillna(False)
    sell_signals = data['Sell_Signal'].shift(1).fillna(False)
    
    data.loc[buy_signals, 'Strategy_Returns'] = data['Log_Returns']
    data.loc[sell_signals, 'Strategy_Returns'] = -data['Log_Returns'] # Go short

    data['Cumulative_P_L'] = data['Strategy_Returns'].cumsum().apply(np.exp)
    
    factor = np.sqrt(252) if TIME_INTERVAL == '1d' else np.sqrt(52)
    
    sharpe_ratio = data['Strategy_Returns'].mean() / data['Strategy_Returns'].std() * factor
    
    downside_returns = data[data['Strategy_Returns'] < 0]['Strategy_Returns']
    downside_std = downside_returns.std()
    sortino_ratio = data['Strategy_Returns'].mean() / downside_std * factor if downside_std > 0 else np.nan

    cumulative_p_l = data['Cumulative_P_L']
    roll_max = cumulative_p_l.cummax()
    drawdown = cumulative_p_l / roll_max - 1.0
    max_drawdown = drawdown.min()

    hit_rate = (data['Strategy_Returns'] > 0).mean()

    print(f"\nBacktesting Metrics:")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"  Sortino Ratio: {sortino_ratio:.2f}")
    print(f"  Maximum Drawdown: {max_drawdown:.2%}")
    print(f"  Overall Hit Rate: {hit_rate:.2%}")

    return data


results = {}
for ticker in YF_TICKERS:
    data, bull_regime, bear_regime = process_index(ticker)
    backtested_data = form_trading_rules_and_backtest(data, bull_regime, bear_regime)
    results[ticker] = backtested_data

if '^GSPC' in results:
    results['^GSPC']['Cumulative_P_L'].plot(title='Cumulative P&L for ^GSPC Strategy')
    plt.show()




--- Processing ^GSPC ---


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_91537/2336944315.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, interval=TIME_INTERVAL)
[*********************100%***********************]  1 of 1 completed


AttributeError: 'DataFrame' object has no attribute 'to_frame'